# 👾 PixelCNN - 텐서플로 확률 라이브러리

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/rickiepark/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/05_autoregressive/03_pixelcnn_md/pixelcnn_md.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

In [ ]:
import sys

# 코랩의 경우 깃허브 저장소로부터 utils.py를 다운로드 합니다.
if 'google.colab' in sys.modules:
    !wget https://raw.githubusercontent.com/rickiepark/Generative_Deep_Learning_2nd_Edition/main/notebooks/utils.py
    !mkdir -p notebooks
    !mv utils.py notebooks
    # output 디렉토리를 만듭니다.
    !mkdir output

이 노트북은 텐서플로 확률 라이브러리를 사용해 패션 MNIST 데이터셋에서 PixelCNN을 훈련합니다.

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, callbacks
import tensorflow_probability as tfp

from notebooks.utils import display

## 0. 파라미터 <a name="parameters"></a>

In [ ]:
IMAGE_SIZE = 32
N_COMPONENTS = 5
EPOCHS = 10
BATCH_SIZE = 128

## 1. 데이터 준비 <a name="prepare"></a>

In [ ]:
# 데이터 로드
(x_train, _), (_, _) = datasets.fashion_mnist.load_data()

In [ ]:
# 데이터 전처리
def preprocess(imgs):
    imgs = np.expand_dims(imgs, -1)
    imgs = tf.image.resize(imgs, (IMAGE_SIZE, IMAGE_SIZE)).numpy()
    return imgs


input_data = preprocess(x_train)

In [ ]:
# 훈련 세트에 있는 샘플 출력하기
display(input_data)

## 2. PixelCNN 만들기 <a name="build"></a>

In [ ]:
# PixelCNN 모델 정의
dist = tfp.distributions.PixelCNN(
    image_shape=(IMAGE_SIZE, IMAGE_SIZE, 1),
    num_resnet=1,
    num_hierarchies=2,
    num_filters=32,
    num_logistic_mix=N_COMPONENTS,
    dropout_p=0.3,
)

# 모델 입력을 정의합니다.
image_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))

# 로그 가능도를 손실 함수로 정의합니다.
log_prob = dist.log_prob(image_input)

# 모델을 정의합니다.
pixelcnn = models.Model(inputs=image_input, outputs=log_prob)
pixelcnn.add_loss(-tf.reduce_mean(log_prob))

## 3. PixelCNN 훈련하기 <a name="train"></a>

In [ ]:
# 모델 컴파일 및 훈련
pixelcnn.compile(
    optimizer=optimizers.Adam(0.001),
)

In [ ]:
tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")


class ImageGenerator(callbacks.Callback):
    def __init__(self, num_img):
        self.num_img = num_img

    def generate(self):
        return dist.sample(self.num_img).numpy()

    def on_epoch_end(self, epoch, logs=None):
        generated_images = self.generate()
        display(
            generated_images,
            n=self.num_img,
            save_to="./output/generated_img_%03d.png" % (epoch),
        )


img_generator_callback = ImageGenerator(num_img=2)

In [ ]:
pixelcnn.fit(
    input_data,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=True,
    callbacks=[tensorboard_callback, img_generator_callback],
)

## 4. 이미지 생성하기 <a name="generate"></a>

In [ ]:
generated_images = img_generator_callback.generate()

In [ ]:
display(generated_images, n=img_generator_callback.num_img)